Testbed for bots, by Arseny, based on Henry's sketch

In [27]:
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json

import draftsimtools as ds

In [28]:
# Load M19 drafts
raw_drafts = ds.load_drafts("../../data/m19_2.csv")

# m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")

In [29]:
with open('../../data/Allsets.json', 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
    
jsonSubset = mtgJSON['M19']['cards']

thisSet = {card['name'] : card for card in jsonSubset}
nameList = pd.DataFrame(thisSet.keys(),columns = ['Name']) # Instead of relying on cvs, get names from json

In [30]:
raw_drafts = ds.fix_commas(nameList, raw_drafts)

In [23]:
# Process the drafts
drafts = ds.process_drafts(raw_drafts)

# Splits into (toy) training and test sets. NOTE: For real training, use all drafts.
#subset_drafts = drafts[5000:5500]
#train, test = train_test_split(subset_drafts, test_size = 0.4)

AttributeError: 'tuple' object has no attribute 'split'

In [ ]:
# Creates a bot that implements current Draftsim logic
draftsim_bot = ds.DraftsimBot(rating_dict)

# Creates a bot that implements current Draftsim logic with SGD optimization
sgd_bot = ds.DraftsimSGDBot(rating_dict)
epochs = 5
for x in range(epochs):
    sgd_bot.sgd_optimization(train, 0.05)
print("Done optimizing SGD bot")

# Gets bot accuracy for some packs
tester = ds.BotTester(test)
tester.evaluate_bots([draftsim_bot, sgd_bot], ["Draftsim2018", "Draftsim2018SGD"])
tester.write_evaluations()